In [ ]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
import pickle

import numpy as np
import matplotlib.pyplot as plt

from itertools import product

from sklearn.model_selection import KFold

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
import sklearn
from numba import jit
import itertools
import scipy
import time
import gc
import copy
import random    
from sklearn.ensemble import BaggingRegressor
from scipy.sparse import csc_matrix
import sys

import matplotlib as mpl
import matplotlib
matplotlib.rcParams.update(matplotlib.rcParamsDefault)



matplotlib.rcParams.update({})
import warnings
warnings.filterwarnings("ignore")

import sys
def load_openml(data_id,ordinal = True, y_label = ''):
    """Load dataset by id from OpenML. If ordinal == True, encode categorical columns 
    via ordinal encoding. If ordinal == False then encode categorical columns with dummy vars.
    """
    
    dataset1 = sklearn.datasets.fetch_openml(data_id = data_id,as_frame = True)
    name = dataset1.details['name']
    X, y = dataset1.data, dataset1.target 
    data = pd.DataFrame(X,columns = dataset1.feature_names)

    if len(y_label) == 0:
        data['y'] = y
    else:
        data['y'] = y[y_label]

    #shuffle index
    data = data.sample(frac = 1)
    y = data['y']
    y = y.astype(float)
    X = data.drop('y',axis = 1)

    #encode categorical columns
    cat = list(set(X.columns) - set(X.select_dtypes(include=np.number).columns.tolist()))
    
    if ordinal == True:
        for col in cat:
            X[col] = X[col].astype('category').cat.codes
            X[col] = X[col].fillna(max(X[col]+1))
        
    elif ordinal == False:
        X = pd.get_dummies(X,columns = cat)
    

    return X,y,name


"""
get np2darray representation of each rule, 1st column = (features ), 2nd column (split values)
all representations are stacked into a list
"""
def get_rule_list(tree_list):
    rule_list = []
    for tree1 in tree_list:
        n_nodes = tree1.tree_.node_count
        children_left = tree1.tree_.children_left
        children_right = tree1.tree_.children_right
        feature = tree1.tree_.feature
        threshold = tree1.tree_.threshold
        value = tree1.tree_.value
        #can also represent via values so adjacent leaf nodes are not the same...
        
        
        branches = list(retrieve_branches(n_nodes, children_left, children_right))
        for b in branches:
  
            rule = np.column_stack((feature[b[:-1]],threshold[b[:-1]]))
            rule_set = set()
            i1 = 0
            for r in rule:
                if b[i1+1] == children_left[b[i1]]:
                    rule_set.add(str(r[0]) +str(r[1])+'L')
                else:
                    rule_set.add(str(r[0]) +str(r[1])+'R')
                i1 = i1 + 1
                
            rule_list.append(rule_set)

    return rule_list


def retrieve_branches(number_nodes, children_left_list, children_right_list):

    # Calculate if a node is a leaf
    is_leaves_list = [(False if cl != cr else True) for cl, cr in zip(children_left_list, children_right_list)]
    
    # Store the branches paths
    paths = []
    
    for i in range(number_nodes):
        if is_leaves_list[i]:
            # Search leaf node in previous paths
            end_node = [path[-1] for path in paths]

            # If it is a leave node yield the path
            if i in end_node:
                output = paths.pop(np.argwhere(i == np.array(end_node))[0][0])
                yield output

        else:
            
            # Origin and end nodes
            origin, end_l, end_r = i, children_left_list[i], children_right_list[i]

            # Iterate over previous paths to add nodes
            for index, path in enumerate(paths):
                if origin == path[-1]:
                    paths[index] = path + [end_l]
                    paths.append(path + [end_r])

            # Initialize path in first iteration
            if i == 0:
                paths.append([i, children_left_list[i]])
                paths.append([i, children_right_list[i]])

def get_uniques(rule_list):
    unique_rules = [rule_list[0]]
    inds = [0]
    ind = 1
    for r in rule_list[1:]:
        
        counter = 0
        for r1 in unique_rules:
            counter = counter + int(r != r1)
        if counter == len(unique_rules):
            unique_rules.append(r)
            inds.append(ind)
            
        ind = ind + 1
            
    counts = []
    for u in unique_rules:
        counter = 0
        for j in rule_list:
            if u == j:
                counter = counter + 1
        counts.append(counter)
    return np.array(unique_rules), np.array(inds),np.array(counts)


def get_tree_matrix_sparse(X,tree1):
    leaf_all = np.where(tree1.tree_.feature < 0)[0]
    leaves_index = tree1.apply(X.values)
    leaves = np.unique(leaves_index)
    values = np.ndarray.flatten(tree1.tree_.value)
    leaves_values = [values[i] for i in leaves_index]
    df = pd.DataFrame(np.column_stack((range(0,len(leaves_index)),leaves_index,leaves_values))
             ,columns = ['instance','node','value'])
    setdiff = list(set(leaf_all) - set(np.unique(leaves_index)))
    toadd = pd.DataFrame(np.column_stack((np.zeros(len(setdiff)),setdiff,np.zeros(len(setdiff)))),
                        columns = ['instance','node','value'])
    df = df.append(toadd)
    matrix_temp = pd.pivot_table(df, index = 'instance',columns = 'node',values = 'value').fillna(0)
    return csc_matrix(matrix_temp.values), matrix_temp.columns.values

def get_rule_matrix_sparse(X,tree_list):
    matrix_full, nodes = get_tree_matrix_sparse(X,tree_list[0])
    node_list = [nodes]
    for tree1 in tree_list[1:]:
        matrix_temp, nodes = get_tree_matrix_sparse(X,tree1)
        node_list.append(nodes)
        matrix_full = scipy.sparse.hstack([matrix_full,matrix_temp])
    return matrix_full.tocsc(), node_list # node list gives the indicies of the nodes in the tree structure

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3



def quantize_X(xTrain,xTest,nbins = 10):
    cols = xTrain.columns
    to_replace = cols[[(len(np.unique(xTrain[c]))>=nbins) | (len(np.unique(xTest[c]))>=nbins)  for c in cols]]
    #pd.qcut(xTrain.rank(method='first')[col, bins)
    for col in to_replace:
        xTrain[col] = pd.qcut(xTrain.rank(method='first')[col],nbins,labels=False)
        xTest[col]= pd.qcut(xTest.rank(method='first')[col],nbins,labels=False)
    return xTrain, xTest
        
"""
get np2darray representation of each rule, 1st column = (features ), 2nd column (split values)
all representations are stacked into a list
"""
def get_rule_list(tree_list):
    rule_list = []
    for tree1 in tree_list:
        n_nodes = tree1.tree_.node_count
        children_left = tree1.tree_.children_left
        children_right = tree1.tree_.children_right
        feature = tree1.tree_.feature
        threshold = tree1.tree_.threshold
        value = tree1.tree_.value
        #can also represent via values so adjacent leaf nodes are not the same...
        
        
        branches = list(retrieve_branches(n_nodes, children_left, children_right))
        for b in branches:
  
            rule = np.column_stack((feature[b[:-1]],threshold[b[:-1]]))
            rule_set = set()
            i1 = 0
            for r in rule:
                if b[i1+1] == children_left[b[i1]]:
                    rule_set.add(str(r[0]) +str(r[1])+'L')
                else:
                    rule_set.add(str(r[0]) +str(r[1])+'R')
                i1 = i1 + 1
                
            rule_list.append(rule_set)

    return rule_list


def retrieve_branches(number_nodes, children_left_list, children_right_list):

    # Calculate if a node is a leaf
    is_leaves_list = [(False if cl != cr else True) for cl, cr in zip(children_left_list, children_right_list)]
    
    # Store the branches paths
    paths = []
    
    for i in range(number_nodes):
        if is_leaves_list[i]:
            # Search leaf node in previous paths
            end_node = [path[-1] for path in paths]

            # If it is a leave node yield the path
            if i in end_node:
                output = paths.pop(np.argwhere(i == np.array(end_node))[0][0])
                yield output

        else:
            
            # Origin and end nodes
            origin, end_l, end_r = i, children_left_list[i], children_right_list[i]

            # Iterate over previous paths to add nodes
            for index, path in enumerate(paths):
                if origin == path[-1]:
                    paths[index] = path + [end_l]
                    paths.append(path + [end_r])

            # Initialize path in first iteration
            if i == 0:
                paths.append([i, children_left_list[i]])
                paths.append([i, children_right_list[i]])

def get_uniques(rule_list):
    unique_rules = [rule_list[0]]
    inds = [0]
    ind = 1
    for r in rule_list[1:]:
        
        counter = 0
        for r1 in unique_rules:
            counter = counter + int(r != r1)
        if counter == len(unique_rules):
            unique_rules.append(r)
            inds.append(ind)
            
        ind = ind + 1
            
    counts = []
    for u in unique_rules:
        counter = 0
        for j in rule_list:
            if u == j:
                counter = counter + 1
        counts.append(counter)
    return np.array(unique_rules), np.array(inds),np.array(counts)


def get_tree_matrix_sparse(X,tree1):
    leaf_all = np.where(tree1.tree_.feature < 0)[0]
    leaves_index = tree1.apply(X.values)
    leaves = np.unique(leaves_index)
    values = np.ndarray.flatten(tree1.tree_.value)
    leaves_values = [values[i] for i in leaves_index]
    df = pd.DataFrame(np.column_stack((range(0,len(leaves_index)),leaves_index,leaves_values))
             ,columns = ['instance','node','value'])
    setdiff = list(set(leaf_all) - set(np.unique(leaves_index)))
    toadd = pd.DataFrame(np.column_stack((np.zeros(len(setdiff)),setdiff,np.zeros(len(setdiff)))),
                        columns = ['instance','node','value'])
    df = df.append(toadd)
    matrix_temp = pd.pivot_table(df, index = 'instance',columns = 'node',values = 'value').fillna(0)
    return csc_matrix(matrix_temp.values), matrix_temp.columns.values

def get_rule_matrix_sparse(X,tree_list):
    matrix_full, nodes = get_tree_matrix_sparse(X,tree_list[0])
    node_list = [nodes]
    for tree1 in tree_list[1:]:
        matrix_temp, nodes = get_tree_matrix_sparse(X,tree1)
        node_list.append(nodes)
        matrix_full = scipy.sparse.hstack([matrix_full,matrix_temp])
    return matrix_full.tocsc(), node_list # node list gives the indicies of the nodes in the tree structure

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3



def quantize_X(xTrain,xTest,nbins = 10):
    cols = xTrain.columns
    to_replace = cols[[(len(np.unique(xTrain[c]))>=nbins) | (len(np.unique(xTest[c]))>=nbins)  for c in cols]]
    #pd.qcut(xTrain.rank(method='first')[col, bins)
    for col in to_replace:
        xTrain[col] = pd.qcut(xTrain.rank(method='first')[col],nbins,labels=False)
        xTest[col]= pd.qcut(xTest.rank(method='first')[col],nbins,labels=False)
        
    return xTrain, xTest
        
import gurobipy as gp
from gurobipy import GRB
from numba import jit
from gurobipy import quicksum
### Entire path functions

@jit(nopython=True)
def get_regression_cost_subgradient(X,y,s,lambda_ridge):
    """
    Must all be numpy arrays
    """
    p = X.shape[1]
    X_sub = X[:,s==1]
    
    alpha_star = y - X_sub@np.linalg.inv(np.diag(np.ones(np.sum(s))/lambda_ridge) + \
                                  np.transpose(X_sub)@X_sub)@np.transpose(X_sub)@y
    
    c = 0.5*y@alpha_star
    
    subgradients = np.zeros(p)
    for j in range(0,p):
        subgradients[j] = -lambda_ridge*0.5 *(X[:,j]@alpha_star)**2
    
    return c, alpha_star , subgradients


def get_stability_path_k(ucounts, nonzero, by = 1):
    start = 0
    end = nonzero
    sorted1 = np.sort(ucounts)

    K_range = []
    while start < len(sorted1):
        K_range.append(np.sum(sorted1[start:end]))
        start = start + 1
        end = end + 1
    K_range = np.flip(np.unique(K_range))
    return K_range[::by]

def epsilon_const_path(X,y, lambda_ridge, nonzero, ucounts, k_range,
                            tol,debias =  False, verbose = False, time_limit = 120, time_limit_gurobi = 60):
    regressor = gp.Model()
    dim = X.shape[1]
    nu = regressor.addVar(name="nu") # Weights
    s_new = regressor.addVars(dim, vtype=GRB.BINARY, name="s_new") 

    regressor.setObjective(nu, GRB.MINIMIZE)
    regressor.addConstr(quicksum(s_new) <= nonzero) # Budget constraint
    regressor.addConstr(nu>=0)


    #### settings
    
    if not verbose:
        regressor.params.OutputFlag = 0
    regressor.params.timelimit = time_limit_gurobi
    regressor.params.mipgap = 0.001    


    #stability_budget = regressor.addVar(name="stability_budget")
    #regressor.addConstr( quicksum((s_new[j])*ucounts[j] for j in range(dim)) >= stability_budget)

    stab_constraint = regressor.addConstr( quicksum((s_new[j])*ucounts[j] for j in range(dim)) >= 0)
    results_w = []
    results_obj = []
    for k in k_range:

        regressor.remove(stab_constraint)
        stab_constraint = regressor.addConstr( quicksum((s_new[j])*ucounts[j] for j in range(dim)) >= k)

        #regressor.setAttr("LB", stability_budget, k)
        #regressor.setAttr("UB", stability_budget, k)

        cost = 10**9
        nu1 = 0


        t1 = time.time()
        while  (nu1 - cost)/cost < -tol:
            regressor.optimize()
            s = np.array([s_new[i].X for i in range(dim)])
            nu1 = nu.X
            s = s.astype(int)
            cost, alpha_star, subgradient = get_regression_cost_subgradient(X,y,s,lambda_ridge)  
            regressor.addConstr(nu >= cost + quicksum([ subgradient[i]*(s_new[i] -s[i]) for i in range(dim)]))
            if (time.time() - t1) > time_limit:
                break
            
            #print(len(regressor.getConstrs()))

        results_obj.append([nu1,cost])
        X_sub = X[:,s==1]
        if debias == True:
            inv = np.linalg.inv(np.transpose(X_sub)@X_sub + np.diag(np.ones(np.sum(s))*0.001))
        else:
            inv = np.linalg.inv(np.transpose(X_sub)@X_sub + np.diag(np.ones(np.sum(s))/lambda_ridge))

        w_sub = inv@np.transpose(X_sub)@y
        w = np.zeros(len(s))
        w[s==1] = w_sub
        results_w.append(w)
        print(k,nu1,cost, time.time()-t1)
        
    return np.array(results_w),results_obj

def r_squared(yTest,pred,yTrain):
    top = np.sum((yTest - pred)**2)
    bottom = np.sum((yTest - np.mean(yTrain))**2)
    return 1 - top/bottom

In [ ]:
### Heuristic Helper Functions

def mcpthresh(w_j,lambda1,gamma):
    if np.abs(w_j) <= lambda1*gamma:
        return soft_threshold(w_j,lambda1)/(1-(1/gamma))
    else:
        return w_j
    
def soft_threshold(w, lambda_):
    return (w / abs(w)) * max(abs(w) - lambda_, 0) if abs(w) > lambda_ else 0

def mcploss(w,lambda1,gamma):
    cond1 = (w <= lambda1*gamma)
    cond2 = (w >= lambda1*gamma)
    mcp = np.zeros(len(w))
    mcp[cond1] = lambda1*w[cond1]- (w[cond1]*w[cond1])/(2*gamma)
    mcp[cond2] = 0.5*gamma*lambda1*lambda1
    return sum(mcp)

def mcpsingle(w_j,lambda1,gamma):
    if np.abs(w_j) <= lambda1*gamma:
        return lambda1*w_j - (w_j*w_j)/(2*gamma)
    else:
        return 0.5*gamma*lambda1*lambda1

def MCP_CD(y,M,lambda1,gamma,threshold= 10**-3,ws = []):
    n = M.shape[0]
    p = M.shape[1]

    loss_sequence = []
    cycle_loss = []
    converged = False
    
    if len(ws) == 0:
        w = np.zeros(p)
    else:
        w = ws
    
    r = y - M@w
    mcp_penalty = mcploss(np.abs(w),lambda1,gamma)
    ind = 0
    while converged == False:
        j = ind%p
        M_j = M[:,j]
        w_j = w[j]

        r = r + M_j*w_j
        mcp_penalty = mcp_penalty - mcpsingle(np.abs(w_j),lambda1,gamma)


        w_sol = M_j@r/(M_j@M_j)
        w[j] = mcpthresh(w_sol,lambda1,gamma)

        r = r - M_j*w[j]
        mcp_penalty = mcp_penalty + mcpsingle(np.abs(w[j]),lambda1,gamma)

        loss = (0.5/n)*r@r + mcp_penalty
        loss_sequence.append(loss)

        ind = ind + 1

        if (j == 0):
            cycle_loss.append(loss)
            if len(cycle_loss) >= 2:
                converged = np.abs(cycle_loss[-1]-cycle_loss[-2])<= threshold
    return w, loss_sequence


#0.5*||y - Mw||_2^2 -    (1-rho)/rho  * sum(ucounts(w!=0)) + lambda_s/rho (sum(w!=0))  
#objective

@jit(nopython=True)
def JIT_cd(j,r, M_j, w, rho,lambda_s, ucounts, n):
    r = r + M_j*w[j]
    w_star = M_j@r/(M_j@M_j)
    obj1 = 0.5*(r-M_j*w_star)@(r-M_j*w_star) + lambda_s/rho - ucounts[j]*(1-rho)/rho
    obj2 = 0.5*r@r
        
        
    if obj2 <= obj1:
        w[j] = 0
    else:
        w[j] = w_star
            
    r = r - M_j*w[j]
    loss = (0.5*rho*r@r - (1-rho)*np.sum(ucounts[w!=0]) + lambda_s*np.sum(w!=0))/n 
    return w,r,loss

def CD_L0(y,M,ucounts, rho,lambda_s, threshold, ws = []):
    n = M.shape[0]
    p = M.shape[1]
    
    if len(ws) == 0:
        w = np.zeros(p)
        r = y
    else:
        w = ws
        r = y - M@w
    
    ind = 0
    loss_sequence = []
    cycle_loss = []
    converged = False
    
    
    while converged == False:
        j = ind % p
        M_j = M[:,j]
        
        w, r,loss = JIT_cd(j,r, M_j, w, rho,lambda_s, ucounts, n)
        
        loss_sequence.append(loss) 
        if (j == 0):
            cycle_loss.append(loss)
            if len(cycle_loss) >= 2:
                converged = np.abs(cycle_loss[-1]-cycle_loss[-2])<= threshold
        ind = ind + 1
    return w, loss_sequence



In [ ]:
import gurobipy as gp
bins = 10
ntrees = 500
non_zero1 = 15
verbose = False
time_limit = 600
time_limit_gurobi = 120
debias = True
tol = 0.05

rho_range = np.arange(0.25,1.0,.025)
rho_range = np.append(rho_range,0.99)
arange = np.flip(np.logspace(0,2,200))
lrange = np.logspace(0,-3,100)
gamma = 1.1

ids = [
195,
505,
560,
690,
519,
196,
1027,
547,
223,
541,
41021,
315,
507,
529,
183,
42570,
405,
294,
287,
503,
308,
558,
227,
189,
296,
201,
216,
537,
574,
344,
215,
564,
42225]
print(len(ids))



In [ ]:
seed = 42
np.random.seed(seed)
for id1 in ids:
    try:

        X1,y1,name = load_openml(id1,ordinal = False, y_label = '')
        lambda_ridge = 0.01
        if X1.shape[0] <= 1000:
            lambda_ridge = 0.05

        print(name,X1.shape,lambda_ridge)
        fire_rules = []
        fire_perfs = []
        sirus_rules = []
        sirus_perfs = []
        rulefit_rules = []
        rulefit_perfs = []

        multiobj_rules = []
        multiobj_perfs = []

        np.random.seed(seed)
        kf = KFold(n_splits=10)
        for i, (train_index, test_index) in enumerate(kf.split(X1)):

            xTrain = X1.iloc[train_index]
            yTrain = y1.iloc[train_index]
            xTest = X1.iloc[test_index]
            yTest = y1.iloc[test_index]
            xTrain = xTrain.fillna(xTrain.median())
            xTest = xTest.fillna(xTrain.median())



            xTrain,xTest = quantize_X(xTrain,xTest,nbins = bins)
            yscaler = preprocessing.StandardScaler()
            yTrain = yscaler.fit_transform(yTrain.values.reshape(-1, 1))
            yTest = yscaler.transform(yTest.values.reshape(-1, 1))
            yTrain = pd.Series(yTrain.flatten())
            yTrain.index = xTrain.index
            yTest = pd.Series(yTest.flatten())
            yTest.index = xTest.index

            np.random.seed(seed)
            rf = RandomForestRegressor(n_estimators = ntrees, n_jobs = -1,
                                   max_features = .3333,max_depth = 2).fit(xTrain,yTrain)

            A, _ = get_rule_matrix_sparse(xTrain,rf.estimators_)
            A_test, _ = get_rule_matrix_sparse(xTest,rf.estimators_)
            A = A.toarray()
            A_test = A_test.toarray()



            tree_list = rf.estimators_
            rule_list = get_rule_list(tree_list)
            unique_rules, uindex,  ucounts = get_uniques(rule_list)
            p_unique = len(unique_rules)

            inds_top = np.argsort(-ucounts)[:non_zero1]

            if (np.sort(ucounts)[::-1][non_zero1-1] == np.sort(ucounts)[::-1][non_zero1]):
                inds_top = inds_top[ucounts[inds_top] != np.sort(ucounts)[::-1][non_zero1]]

            non_zero = len(inds_top)
            sirus = list(np.array(unique_rules)[inds_top])
            sirus_rules.append(sirus)

            A_u = A[:,uindex]
            A_test_u = A_test[:,uindex]

            A1 = A_u[:,inds_top]
            A_test1 = A_test_u[:,inds_top]
            ridge = 0.001
            alpha = np.dot((np.dot(np.linalg.inv(np.dot(A1.T,A1) + np.diag(np.repeat(ridge,len(A1[0])))),A1.T)),yTrain)
            sirus_perfs.append(r_squared(yTest,A_test1@alpha,yTrain))

            s1 = preprocessing.StandardScaler()
            M = s1.fit_transform(A_u)
            M_test = s1.transform(A_test_u)

            del A
            del A_test
            del A1
            del A_test1
            gc.collect()


            ### run multi_obj_heuristic 

            w_all = []
            for rho in rho_range:
                print('rho:', rho)
                ws = []
                coef_heuristic = []
                for alpha_lasso in arange:
                    w, l2 = CD_L0(yTrain.values,M,ucounts, rho,alpha_lasso, 10**-6, ws = ws)
                    coef_heuristic.append(w)
                    ws = copy.deepcopy(w)

                heuristic_best = np.array(coef_heuristic)[[sum(w!=0)<=non_zero for w in coef_heuristic]][-1]
                w_all.append(heuristic_best)


            mobj_temp = []
            mobj_temp_rules = []
            for w in w_all:
                mobj_temp.append(r_squared(yTest,M_test@w,yTrain))
                mobj_temp_rules.append(list(np.array(unique_rules)[w!=0]))

            multiobj_perfs.append(mobj_temp)
            multiobj_rules.append(mobj_temp_rules)
            
            coef_MCP = []
            ws_MCP = []
            for lambda1 in lrange:
                w_MCP, ls = MCP_CD(yTrain.values,M,lambda1,gamma,10**-3,ws = ws_MCP)
                coef_MCP.append(w_MCP)
                ws_MCP = w_MCP.copy()

            fire_best = np.array(coef_MCP)[[sum(w!=0)<=non_zero for w in coef_MCP]][-1]
            fire_error = r_squared(yTest,M_test@fire_best,yTrain)
            fire_rules.append(list(np.array(unique_rules)[fire_best!=0]))
            fire_perfs.append(fire_error)

            print(i)

        res_dict = {}
        res_dict['sirus_rules'] =  sirus_rules
        res_dict['sirus_perfs'] =  sirus_perfs
        res_dict['multiobj_heuristic_rules'] =  multiobj_rules
        res_dict['multiobj_heuristic_perfs'] =  multiobj_perfs
        name1 = name +'_' + str(id1) +'_trees_' + str(ntrees) + '_ridge_' + \
                str(lambda_ridge) + '_tol_' + str(tol) +'_debias_' + str(debias) +'.pickle'

        name1 = 'results_heuristics_granular/pickles/' + name1

        with open(name1, 'wb') as handle:
            pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


        multiobj_perfs = np.array(multiobj_perfs)
        multiobj_perf_means = multiobj_perfs.mean(axis = 0)

        multiobj_stability = []
        for i in range(0,len(multiobj_rules[0])):
            temp = []
            for i1 in range(0,len(multiobj_rules)):
                for i2 in range(i1+1,len(multiobj_rules)):
                    rules1 = list(get_uniques(multiobj_rules[i1][i])[0])   
                    rules2 = list(get_uniques(multiobj_rules[i2][i])[0])
                    temp.append(2*len(intersection(rules1,rules2))/(len(rules1)+len(rules2)))
            multiobj_stability.append(temp)
        multiobj_stability = np.array(multiobj_stability)
        multiobj_stability_means = multiobj_stability.mean(axis = 1)
        
        fire_stability = []
        for i1 in range(0,len(fire_rules)):
            for i2 in range(i1+1,len(fire_rules)):
                rules1 = list(get_uniques(fire_rules[i1])[0])   
                rules2 = list(get_uniques(fire_rules[i2])[0])
                fire_stability.append(2*len(intersection(rules1,rules2))/(len(rules1)+len(rules2)))


        sirus_stability = []
        for i1 in range(0,len(sirus_rules)):
            for i2 in range(i1+1,len(sirus_rules)):
                rules1 = list(get_uniques(sirus_rules[i1])[0])   
                rules2 = list(get_uniques(sirus_rules[i2])[0])
                sirus_stability.append(2*len(intersection(rules1,rules2))/(len(rules1)+len(rules2)))

        print('fire', np.mean(fire_stability), 1-np.mean(fire_perfs))    
        
        plt.scatter(1-np.mean(fire_perfs), np.mean(fire_stability), label = 'fire', color = 'green')
        plt.scatter(1-multiobj_perf_means[1:],multiobj_stability_means[1:],label = 'multiobj', alpha = .3)
        plt.scatter(1-multiobj_perf_means[0],multiobj_stability_means[0],label = 'multiobj_stability', color = 'C0',alpha = .4)

        temp_list = list(range(1,len(multiobj_perf_means[1:]) + 1))
        for i in range(len(multiobj_perf_means[1:],)):
            plt.annotate(temp_list[i], (1-multiobj_perf_means[1:][i], multiobj_stability_means[1:][i]))


        plt.scatter(1-np.mean(sirus_perfs),np.mean(sirus_stability),label = 'sirus', color = 'red')
        plt.legend()

        plt.xlabel('1 - R^2')
        plt.ylabel('Stability')
        name2 = 'results_heuristics_granular/plots/' + name +'_' + str(id1) +'_trees_' + str(ntrees) + '_ridge_' + \
                str(lambda_ridge) + '_tol_' + str(tol) +'_debias_' + str(debias) +'.pdf'
        plt.title(name2)
        plt.savefig(name2)
        plt.show()
    except:
        print(name)

    
